In [96]:
import pandas as pd

In [124]:
hotels = pd.read_csv('hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [98]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  


In [99]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  


In [100]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  

In [101]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
#print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

Небольшой бонус:


In [102]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)

In [103]:
num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts', ...]

# категориальные признаки
cat_cols = ['average_score', 'lat', ...]

In [104]:
from sklearn.feature_selection import f_classif # anova


In [105]:
hotels['hotel_name'].nunique()

1492

In [106]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_date'].min()


Timestamp('2015-08-04 00:00:00')

In [107]:
hotels['review_date'].max()

Timestamp('2017-08-03 00:00:00')

In [108]:
unique = []
a = hotels['tags']


for j in a:
    
    if j in unique:
        continue
    else:
        unique.append(j)
        
display(len(unique))

display(unique)

47135

["[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']",
 "[' Business trip ', ' Couple ', ' Standard Double Room ', ' Stayed 1 night ']",
 "[' Leisure trip ', ' Solo traveler ', ' Modern Double Room Echo ', ' Stayed 3 nights ', ' Submitted from a mobile device ']",
 "[' Leisure trip ', ' Solo traveler ', ' Standard Room with 1 Double Bed ', ' Stayed 1 night ']",
 "[' Business trip ', ' Couple ', ' Standard Double or Twin Room ', ' Stayed 6 nights ']",
 "[' Business trip ', ' Solo traveler ', ' Superior Twin Room ', ' Stayed 1 night ', ' Submitted from a mobile device ']",
 "[' Leisure trip ', ' Couple ', ' Double Room Non Smoking ', ' Stayed 1 night ', ' Submitted from a mobile device ']",
 "[' Leisure trip ', ' Couple ', ' Superior Double Room ', ' Stayed 4 nights ']",
 "[' Leisure trip ', ' Couple ', ' Double Room ', ' Stayed 1 night ']",
 "[' Leisure trip ', ' Couple ', ' Twin Hilton Guest Room ', ' Stayed 1 night ', ' Submitted 

In [109]:
import re # дополнительная библиотека для очистки строки, созданной из массива

dict_of_tags = dict()

for i in range(hotels.shape[0]):
  tags = re.sub("['|\[|\]]", "", hotels['tags'].iloc[i]).split(' , ')
  tags = [tag.replace(' ', '', 1) for tag in tags]
  tags[-1] = tags[-1][:-1]
  for tag in tags:
    if tag not in dict_of_tags:
      dict_of_tags[tag] = 1
    else:
      dict_of_tags[tag] += 1   

display(len(dict_of_tags))

2368

In [110]:
#display(dict_of_tags)
#n = 0
#for Key, Value in dict_of_tags.items():
    #display(Key,Value)
#    if Key[Value] > Key[Value] + 1:
#        n = Value
        
max_znach = max(dict_of_tags.values())
display(max_znach)
list_values = list(dict_of_tags.items())
list_values.sort(key=lambda i: i[1], reverse=True)

#display(list_values)
print(list_values[0])

313593

('Leisure trip', 313593)


In [111]:
#display(unique)
#if 'Stayed' in unique:
#display(hotels['tags'])




In [112]:
import pandas as pd
import numpy as np
import re



def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

hotels['tags_n'] = hotels['tags'].apply(rev_func)


def find_stayed(row):
    
    for tag in row:
        if 'Stayed' in tag:
            return tag.split()[1]
    return 0

hotels['tag_stayed_cnt'] = hotels['tags_n'].apply(find_stayed)



#display(hotels['tag_stayed_cnt'].sort_values())
display(hotels['tag_stayed_cnt'].value_counts())

tag_stayed_cnt
1     145373
2     100263
3      72000
4      35748
5      15611
6       7399
7       5549
8       1910
9        966
10       663
11       306
12       217
14       184
13       174
0        146
15        87
16        38
17        27
18        24
19        23
21        19
20        17
27        10
30        10
22         8
28         7
26         6
23         6
24         5
25         4
29         3
Name: count, dtype: int64

In [113]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline





hotels.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 19 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               386803 non-null  object        
 1   additional_number_of_scoring                386803 non-null  int64         
 2   review_date                                 386803 non-null  datetime64[ns]
 3   average_score                               386803 non-null  float64       
 4   hotel_name                                  386803 non-null  object        
 5   reviewer_nationality                        386803 non-null  object        
 6   negative_review                             386803 non-null  object        
 7   review_total_negative_word_counts           386803 non-null  int64         
 8   total_number_of_reviews                     386803 non-null  int64        

In [121]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)
hotels.drop(columns='review_date') 


# # заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

hotels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 10 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   additional_number_of_scoring                386803 non-null  int64         
 1   review_date                                 386803 non-null  datetime64[ns]
 2   average_score                               386803 non-null  float64       
 3   review_total_negative_word_counts           386803 non-null  int64         
 4   total_number_of_reviews                     386803 non-null  int64         
 5   review_total_positive_word_counts           386803 non-null  int64         
 6   total_number_of_reviews_reviewer_has_given  386803 non-null  int64         
 7   reviewer_score                              386803 non-null  float64       
 8   lat                                         386803 non-null  float64      

In [122]:
RANDOM_SEED = 42

X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 
from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [123]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>)